## Run rapid-mcdr over all trajectory files along the transect in Pacific Ocean
- Perform baseline and sensitivity simulations (from Suselj et al., 2024)
- Please do not save more than 2 full simulation results, otherwise you will run out of space.
- replace `path_out` with path to your local workspace
- IF YOU DECIDE TO RUN NEW EXPERIMENTS, PLEASE CONSULT ALEX AND/OR KAY ON HOW AND WHERE TO SAVE OUTPUT DATA!


In [1]:
import xarray as xr
import numpy as np
import os
# rapid-mcdr model
from model_v1 import rapid_mcdr_model

### Define paths to model inputs and outputs

In [2]:
# paths to input and output datasets

path_in='/home/jovyan/shared/NASA_Summer_School_2024/marine_co2/materials/data/rapid_mcdr_inputs/'

# SET PATH TO YOUR LOCAL DIRECTORY
path_out='/home/jovyan/shared/NASA_Summer_School_2024/marine_co2/materials/data/rapid_mcdr_outputs/'
# list file numbers - it's fine to include files that do not exist 
num_files=range(150)

### Define function to run rapid-mCDR over all files in `path_in`
- Each file in `path_in` includes a time series of rapid-mCDR forcing for a single location along the transect.

In [3]:
def run_rapid_mcdr_model(num_files=range(8),season='yearly',no_seaice=False,zero_w=False,zero_diff=False):
    results=[]

    for nf in num_files:
       # display(nf)
        num_file=str(nf)
        data_path=path_in+'model_forcing_data_daily_'+str(num_file)+'.nc'
        if os.path.exists(data_path):
            
            forc_dta=xr.load_dataset(data_path)

            forc_dta['area']=(('time'),np.zeros(len(forc_dta.time)))
            forc_dta['alk_forcing']=(('time'),np.zeros(len(forc_dta.time)))

            # the amount of Alk forcing mol/s and area 
            # -- rapid-mcdr is linear to those variables and the results will not be sensitive to the values
            alkforc=392098334
            forc_dta['area'][:]=267000000000
            
            if season=='yearly':
                forc_dta['alk_forcing'][0:365]=alkforc
            elif season=='january':
                forc_dta['alk_forcing'][0:31]=alkforc
            elif season=='july':
                forc_dta['alk_forcing'][182:182+31]=alkforc    

            if zero_diff:
                forc_dta['k_diff']=forc_dta['k_diff']*0     
            if zero_w:    
                forc_dta['w']=forc_dta['w']*0    
            if no_seaice:
                forc_dta['siarea']=forc_dta['siarea']*0         
                
            res=rapid_mcdr_model(forc_dta)
            res['efficiency']=(-res['f_co2']/res.alk_forcing.sum()*res.area/res.cdr_potential).cumsum(dim='time')
            forc_dta['potential']=-(forc_dta.dpco2_ov_dalk/forc_dta.dpco2_ov_ddic)
            
            # rename forcing data
            forc_dta=forc_dta.rename({'dic':'dic_forc','alk':'alk_forc'})
            results_one=xr.merge([res,forc_dta])
            results_one=results_one.expand_dims(lat_sim=[forc_dta.lat.mean()])

            if not results:
                results=results_one
            else:
                results=xr.concat([results,results_one],dim='lat_sim')
        else:
            print('data does not exist: '+data_path)
    return results   


## Run rapid-mcdr for different deployments and we save results as netcdf files

Output is a single file per deployment strategy. Here we run, the following experiments:

      1. OAE deployed from January 1 1995 -- December 31 1995 ... results are saved in file `rapid_mcdr_results.ipynb`
      2. OAE deployed from January 1 1995 -- January 31 1995 ...  results are saved in file `rapid_mcdr_results_jan.ipynb`
      3. OAE deployed from July 1 1995 -- July 31 1995 ...  results are saved in file `rapid_mcdr_results_jul.ipynb`

### PLEASE DO NOT RERUN THESE EXPERIMENTS - YOU WILL RUN OUT OF STORAGE SPACE

In [4]:
# compute and save results to file    
results=run_rapid_mcdr_model(num_files=num_files)
results.to_netcdf(path_out+'rapid_mcdr_results.nc')

# set w to zero
#results_nodiff=run_rapid_mcdr_model(num_files=num_files,zero_diff=True)
#results_nodiff.to_netcdf(path_out+'rapid_mcdr_results_zero_diff.nc')

#  set diffusivity to zero
#results_now=run_rapid_mcdr_model(num_files=num_files,zero_w=True)
#results_now.to_netcdf(path_out+'rapid_mcdr_results_zero_w.nc')

# set sea-ice to zero
#results_noseaice=run_rapid_mcdr_model(num_files=num_files,no_seaice=True)
#results_noseaice.to_netcdf(path_out+'rapid_mcdr_results_zero_seaice.nc')

# january and july deployment
results_jan=run_rapid_mcdr_model(num_files=num_files,season='january')
results_jan.to_netcdf(path_out+'rapid_mcdr_results_jan.nc')
results_jul=run_rapid_mcdr_model(num_files=num_files,season='july')
results_jul.to_netcdf(path_out+'rapid_mcdr_results_july.nc')

data does not exist: data/rapid_mcdr_inputs/model_forcing_data_daily_0.nc
data does not exist: data/rapid_mcdr_inputs/model_forcing_data_daily_1.nc
data does not exist: data/rapid_mcdr_inputs/model_forcing_data_daily_2.nc
data does not exist: data/rapid_mcdr_inputs/model_forcing_data_daily_136.nc
data does not exist: data/rapid_mcdr_inputs/model_forcing_data_daily_137.nc
data does not exist: data/rapid_mcdr_inputs/model_forcing_data_daily_138.nc
data does not exist: data/rapid_mcdr_inputs/model_forcing_data_daily_139.nc
data does not exist: data/rapid_mcdr_inputs/model_forcing_data_daily_140.nc
data does not exist: data/rapid_mcdr_inputs/model_forcing_data_daily_141.nc
data does not exist: data/rapid_mcdr_inputs/model_forcing_data_daily_142.nc
data does not exist: data/rapid_mcdr_inputs/model_forcing_data_daily_143.nc
data does not exist: data/rapid_mcdr_inputs/model_forcing_data_daily_144.nc
data does not exist: data/rapid_mcdr_inputs/model_forcing_data_daily_145.nc
data does not exis